<h1> IBM Applied Data Science Capstone Course: Final Notebook 

<h3> <u> Structure of the Noteboook </u> </h3> 

    1. Importing the Libaries
    2. Data Collection from Wikipedia and creation of New DataFrame 
    3. Getting Geographical coordinates of Neighbourhoods 
    4. Visualization of Neighbourhoods
    5. Explore Neighbourhoods using Foursquare API
    6. Analyze Each Neighbourhood
    7. Cluster Neighbourhoods
    8. Examine Clusters
    9. Result

<h3> 1. Importing the Libaries 

In [1]:
import pandas as pd 
import numpy as np 

!pip install beautifulsoup4
!pip install lxml
!pip install html5lib

import requests 
from bs4 import BeautifulSoup 

In [2]:
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
!pip install geocoder
import geocoder
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

<h3> 2. Data Collection from Wikipedia and creation of New DataFrame 
 

In [3]:
# I extracted the data regarding postal codes, boroughs and neighbourhood of Toronto from Wikipedia
# using GET request and Beautifulsoup and created a dataframe 

page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
contents = page.content

soup = BeautifulSoup(contents, 'lxml') #used BeautifulSoup to retrieve the data form wikipedia
links =soup.find_all("a")

a = soup.find_all("table")[0] 
data = pd.read_html(str(a))
df = data[0]

print(df.shape)
print(df.head())

(288, 3)
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


In [4]:
# i got rid of the rows where Borough = Not assigned 

df=df[df.Borough != 'Not assigned']
df = df.sort_values(by=['Postcode','Borough'])

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

print(df.shape)
print(df.head())

(211, 3)
  Postcode      Borough   Neighbourhood
0      M1B  Scarborough           Rouge
1      M1B  Scarborough         Malvern
2      M1C  Scarborough  Highland Creek
3      M1C  Scarborough      Rouge Hill
4      M1C  Scarborough      Port Union


In [5]:
# i got rid of the rows where Borough = Not assigned 
# and replaced the Neighbourhood = Not assigned with the corresponding Borough value

df=df[df.Borough != 'Not assigned']
df = df.sort_values(by=['Postcode','Borough'])

df['Neighbourhood'].loc[df['Neighbourhood'] == 'Not assigned'] = df['Borough']

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

print(df.shape)
print(df.head())


(211, 3)
  Postcode      Borough   Neighbourhood
0      M1B  Scarborough           Rouge
1      M1B  Scarborough         Malvern
2      M1C  Scarborough  Highland Creek
3      M1C  Scarborough      Rouge Hill
4      M1C  Scarborough      Port Union


<h3> 3. Getting Geographical coordinates of Neighbourhoods
 

In [6]:

# i define a function to get coordinates where i create a loop to get it for all Neighbourhoods
def get_latlng(neighbourhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, ON'.format(neighbourhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coordinates = [ get_latlng(neighbourhood) for neighbourhood in df["Neighbourhood"].tolist() ]

coordinates

[[43.807660000000055, -79.17404999999997],
 [43.81024000000008, -79.22033999999996],
 [43.789480000000026, -79.17613999999998],
 [43.78716000000003, -79.13251999999994],
 [43.77897000000007, -79.13108999999997],
 [43.74953000000005, -79.18991999999997],
 [43.78255000000007, -79.20522999999997],
 [43.76343000000003, -79.17819999999995],
 [43.767480000000035, -79.22828999999996],
 [43.74772769666329, -79.23517417124673],
 [43.73855000000003, -79.21693999999997],
 [43.78144256372015, -79.29834662238574],
 [43.736290000000054, -79.27319999999997],
 [43.72587000000004, -79.26228999999995],
 [43.71682000000004, -79.29959999999994],
 [43.72599000000008, -79.28326999999996],
 [43.698830000000044, -79.27911999999998],
 [43.726420000000076, -79.23296999999997],
 [43.70380251560437, -79.25063973450567],
 [43.67980579433121, -79.28677789503156],
 [43.68436000000003, -79.27589999999998],
 [43.70380251560437, -79.25063973450567],
 [43.75533000000007, -79.27745999999996],
 [43.775600000000054, -79.26

In [7]:
# i create new dataframe where i store the coordinates as Latitude and Longitude

df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [8]:
# here i create two new columns that contain coordinates of neighbourhoods in my main dataframe 

df['Latitude'] = df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']

print(df.shape)
print(df.head())

(211, 5)
  Postcode      Borough   Neighbourhood  Latitude  Longitude
0      M1B  Scarborough           Rouge  43.80766  -79.17405
1      M1B  Scarborough         Malvern  43.81024  -79.22034
2      M1C  Scarborough  Highland Creek  43.78948  -79.17614
3      M1C  Scarborough      Rouge Hill  43.78716  -79.13252
4      M1C  Scarborough      Port Union  43.77897  -79.13109


In [9]:
#lets create csv file and store the data there

df.to_csv("Capstone_df.csv", index=False)

<h3> 4. Visualization of Neighbourhoods

In [10]:
# get the coordinates of Toronto
address = 'Toronto, On'

geolocator = Nominatim(user_agent="user")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON 43.653963, -79.387207.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [12]:
# Lets save the map as HTML file
map_toronto.save('map_toronto.html')

<h3> 5. Explore Neighbourhoods using Foursquare API

In [1]:
# define Foursquare Credentials and Version
CLIENT_ID = 'Your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'Your Foursquare Secret' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Your Foursquare ID
CLIENT_SECRET:Your Foursquare Secret


<b> Now, let's get the top 100 venues that are within a radius of 1500 meters.<b>

In [14]:
radius = 1500
LIMIT = 100

venues = []

for lat, long, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [15]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names. 
# i will not use PostalCode and Borough data as i do not need it for my analysis
venues_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

print(venues_df.shape)
venues_df.head()

(13736, 7)


,Neighbourhood,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.80766,-79.17405,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
1,Rouge,43.80766,-79.17405,Orangutan Exhibit,43.818413,-79.182548,Zoo Exhibit
2,Rouge,43.80766,-79.17405,Pizza Pizza,43.806613,-79.178445,Pizza Place
3,Rouge,43.80766,-79.17405,Dean Park,43.804364,-79.169159,Park
4,Rouge,43.80766,-79.17405,Highland Creek Overpass,43.805062,-79.173280,Bus Station


<b> Let's check how many venues were returned for each neighorhood <b>

In [16]:
venues_df.groupby(["Neighbourhood"]).count()

,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,74,74,74,74,74,74
Agincourt North,40,40,40,40,40,40
Albion Gardens,54,54,54,54,54,54
Alderwood,64,64,64,64,64,64
Bathurst Manor,55,55,55,55,55,55
Bathurst Quay,100,100,100,100,100,100
Bayview Village,62,62,62,62,62,62
Beaumond Heights,19,19,19,19,19,19


<b> Let's find out how many unique categories are there and what are they. 

In [17]:
print('There are {} uniques categories.'.format(len(venues_df['Venue Category'].unique())))

print(venues_df['Venue Category'].unique())

There are 354 uniques categories.
['Zoo Exhibit' 'Pizza Place' 'Park' 'Bus Station' 'Fast Food Restaurant'
 'Trail' 'Other Great Outdoors' 'Tram Station' 'Café' 'Pharmacy'
 'Sandwich Place' 'Skating Rink' 'Chinese Restaurant' 'Grocery Store'
 'Bubble Tea Shop' 'Salon / Barbershop' 'Sports Bar' 'Italian Restaurant'
 'Breakfast Spot' 'Athletics & Sports' 'Neighborhood' 'Burger Joint'
 'Coffee Shop' 'Gym' 'Restaurant' 'Hotel' 'Cosmetics Shop' 'Beach' 'Pub'
 'Bank' 'Supermarket' 'Beer Store' 'Food & Drink Shop' 'Pet Store'
 'Train Station' 'Convenience Store' 'Japanese Restaurant' 'Liquor Store'
 'Yoga Studio' 'Storage Facility' 'Baseball Field' 'Moving Target'
 'Bus Line' 'Discount Store' 'Fried Chicken Joint' 'Intersection'
 'Smoothie Shop' 'Gym / Fitness Center' 'Greek Restaurant'
 'Gymnastics Gym' 'Mexican Restaurant' 'Automotive Shop'
 'Asian Restaurant' 'Bar' 'Indian Restaurant' 'Caribbean Restaurant'
 'Music Store' 'Hakka Restaurant' 'Paper / Office Supplies Store'
 'Vietnamese Rest

In [18]:
# check if the results contain "Bubble Tea Shop"
"Bubble Tea Shop" in venues_df['Venue Category'].unique()

True

<h3> 6. Analyze Each Neighbourhood

In [19]:
# one hot encoding<h3> 6. Analyze Each Neighbourhood
df_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

print(df_onehot.shape)
df_onehot.head()

(13736, 355)


,Neighbourhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# now i group rows by neighbourhood and by taking the mean of the frequency of occurrence 
# of each category

df_grouped = df_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(df_grouped.shape)
df_grouped

(209, 355)


,Neighbourhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Adelaide,0.0,0.000000,0.0,0.00,0.0,0.010000,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000
1,Agincourt,0.0,0.000000,0.0,0.00,0.0,0.013514,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000
2,Agincourt North,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Albion Gardens,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Alderwood,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000
5,Bathurst Manor,0.0,0.000000,0.0,0.00,0.0,0.018182,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bathurst Quay,0.0,0.000000,0.0,0.01,0.0,0.010000,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000
7,Bayview Village,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Beaumond Heights,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Bedford Park,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.00,0.0,0.014493,0.000000,0.000000,0.000000,0.000000,0.000000


<b> Create a new DataFrame for 'Bubble Tea Shop' data only

In [22]:
df_mall = df_grouped[["Neighbourhoods","Bubble Tea Shop"]]
df_mall.head()

,Neighbourhoods,Bubble Tea Shop
0,Adelaide,0.000
1,Agincourt,0.000
2,Agincourt North,0.025
3,Albion Gardens,0.000
4,Alderwood,0.000


<h3> 7. Cluster Neighbourhoods

Run k-means to cluster the neighbourhoods in Toronto into 5 clusters.

In [23]:
# set number of clusters
clusters = 5

df_clustering = df_mall.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 2], dtype=int32)

In [24]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood.
df_merged = df_mall.copy()

# add clustering labels
df_merged["Cluster Labels"] = kmeans.labels_

In [25]:
df_merged.head()

,Neighbourhoods,Bubble Tea Shop,Cluster Labels
0,Adelaide,0.000,0
1,Agincourt,0.000,0
2,Agincourt North,0.025,4
3,Albion Gardens,0.000,0
4,Alderwood,0.000,0


In [26]:
# merge df and df_merged to add latitude/longitude for each neighbourhood
df_merged = df_merged.join(df.set_index("Neighbourhood"), on="Neighbourhoods")

In [27]:
df_merged = df_merged.drop(["Postcode", "Borough"], axis = 1)
df_merged.head()

,Neighbourhoods,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,Adelaide,0.000,0,43.645090,-79.40162
1,Agincourt,0.000,0,43.785980,-79.28020
2,Agincourt North,0.025,4,43.809300,-79.26707
3,Albion Gardens,0.000,0,43.695114,-79.28603
4,Alderwood,0.000,0,43.604960,-79.54116


In [28]:
# sort the results by Cluster Labels
print(df_merged.shape)
df_merged.sort_values(["Cluster Labels"], inplace=True)
df_merged.head()

(211, 5)


,Neighbourhoods,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,Adelaide,0.0,0,43.645090,-79.401620
126,Old Mill South,0.0,0,43.649261,-79.496732
127,Oriole,0.0,0,43.693643,-79.401812
128,Parkdale,0.0,0,43.639130,-79.450730
129,Parkdale Village,0.0,0,43.641321,-79.451436


<b> Finally, let's visualize the resulting clusters

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i+x+(i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhoods'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
# lets save the map as HTML file
map_clusters.save('map_clusters.html')

<h3> 8. Examine Clusters

<b> Cluster 0 

In [32]:
df_merged.loc[df_merged['Cluster Labels'] == 0].head()

,Neighbourhoods,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,Adelaide,0.0,0,43.645090,-79.401620
126,Old Mill South,0.0,0,43.649261,-79.496732
127,Oriole,0.0,0,43.693643,-79.401812
128,Parkdale,0.0,0,43.639130,-79.450730
129,Parkdale Village,0.0,0,43.641321,-79.451436


In [33]:
df_merged.loc[df_merged['Cluster Labels'] == 1].head()

,Neighbourhoods,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
200,Willowdale West,0.080000,1,43.77094,-79.42785
198,Willowdale,0.071429,1,43.78227,-79.42813


In [34]:
df_merged.loc[df_merged['Cluster Labels'] == 2].head()

,Neighbourhoods,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
142,Rosedale,0.010000,2,43.677070,-79.388980
66,Harbourfront East,0.010000,2,43.641424,-79.376131
196,Wexford,0.014493,2,43.750120,-79.299600
106,Maryvale,0.013699,2,43.750830,-79.307690
9,Bedford Park,0.014493,2,43.728300,-79.399180


In [35]:
df_merged.loc[df_merged['Cluster Labels'] == 3].head()

,Neighbourhoods,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
117,Newtonbrook,0.050633,3,43.787300,-79.409830
133,Princess Gardens,0.040000,3,43.771748,-79.397347
102,Malvern,0.058824,3,43.810240,-79.220340


In [36]:
df_merged.loc[df_merged['Cluster Labels'] == 4].head()

,Neighbourhoods,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
134,Queen's Park,0.020000,4,43.66253,-79.39017
148,Runnymede,0.021739,4,43.68341,-79.48291
98,Lawrence Park,0.017544,4,43.72554,-79.40232
2,Agincourt North,0.025000,4,43.80930,-79.26707
148,Runnymede,0.021739,4,43.68341,-79.48291


<h3> 9. Result



From clustering we observe that the highest number of Bubble Tea Shops are concentrated in Clusters <b> 1 </b> and <b> 3 </b>, which make those cluster areas competitive and non-profitable for new investors. On the other hand, Cluster <b> 0 </b> almost does not have any Bubble Tea Shops. Considering that territorial area of Cluster <b> 0 </b> is large ( **Red** colored in map) we can suggest that opening new Bubble Taa Shop in that area may be beneficial and profitable for investors, as investor can capture large market share and take advantage of low competition. 